In [1]:
import ee
import geemap
ee.Authenticate()
ee.Initialize(project='andy-geog761')



In [2]:
import geemap
# --- Define your specific parameters ---
# Point of Interest: Latitude and Longitude
# Example: Coordinates for the Eiffel Tower
latitude = -37.1339
longitude = 174.7065
poi = ee.Geometry.Point(longitude, latitude)


# Specific Date of Interest
# Note: GEE filters are [start, end), so we use a one-day range.
date_of_interest = '2023-01-26'
start_date = date_of_interest
end_date = ee.Date(date_of_interest).advance(20, 'day').format('YYYY-MM-dd')

# --- 1. Get Sentinel-1 Data (Radar) ---
# Load the Sentinel-1 GRD image collection
s1_collection = ee.ImageCollection('COPERNICUS/S1_GRD') \
    .filterBounds(poi) \
    .filterDate(start_date, end_date) \
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')) \
    .select(['VV', 'VH'])

# Get the first available image
s1_image = s1_collection.first()

# --- 2. Get Sentinel-2 Data (Optical RGB) ---
# Load the Sentinel-2 Surface Reflectance image collection
s2_collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterBounds(poi) \
    .filterDate(start_date, end_date) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))# Filter for low cloud cover

# Get the first available image and select RGB bands
s2_image = s2_collection.select(['B4', 'B3', 'B2']).first() # B4=Red, B3=Green, B2=Blue

# --- 3. Define Visualization Parameters ---

# For Sentinel-1 (Radar): We'll visualize the VV polarization in grayscale.
# Min/max values are in dB and chosen to provide good contrast for land features.
s1_vis_params = {
    'bands': ['VV'],
    'min': -25,
    'max': 0
}

# For Sentinel-2 (Optical): Create a true-color composite (RGB).
# Min/max are surface reflectance values. 0-3000 is a common range for good contrast.
s2_vis_params = {
    'bands': ['B4', 'B3', 'B2'], # Red, Green, Blue
    'min': 0,
    'max': 3000,
    'gamma': 1.4
}

# --- 4. Create and Display the Map ---
# Create an interactive map centered on your point
Map = geemap.Map(center=[latitude, longitude], zoom=12)

# Add the satellite images as layers to the map
# Check if images exist before adding them
if s1_image.getInfo():
    Map.addLayer(s1_image, s1_vis_params, 'Sentinel-1 (Radar VV)')
else:
    print("❌ No Sentinel-1 image found to display.")

if s2_image.getInfo():
    Map.addLayer(s2_image, s2_vis_params, 'Sentinel-2 (RGB)')
else:
    print("❌ No Sentinel-2 image found to display.")

# Add a marker for your point of interest
Map.add_marker(location=[latitude, longitude])

# Add a layer controller to toggle layers on and off
Map.addLayerControl()

# Display the map in the notebook
Map

Map(center=[-37.1339, 174.7065], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Search…

In [7]:


# Import the necessary libraries
import ee
import geemap
import numpy as np



# --- 2. Define the 12 Sentinel-2 Bands ---
# These are the main spectral bands for analysis.
S2_BANDS = [
    'B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12'
]

# --- 3. Get the Sentinel-2 Image with ALL 12 Bands ---
s2_image_12_band = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterBounds(poi) \
    .filterDate(start_date, end_date) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
    .select(S2_BANDS) \
    .first()

# --- 4. Convert the 12-Band Image to a NumPy Array ---
if s2_image_12_band.getInfo():
    print(f"✅ Found Sentinel-2 image with bands: {s2_image_12_band.bandNames().getInfo()}")

    # Define the region to extract
    region_of_interest = poi.buffer(1280).bounds()

    # Convert the composite image to a NumPy array
    numpy_array_12_band = geemap.ee_to_numpy(s2_image_12_band, region=region_of_interest)

    print(f"\nShape of the new array: {numpy_array_12_band.shape}")
    print("This shape matches the model's expected input of 12 channels.")

else:
    print("❌ Could not find a Sentinel-2 image for the specified date.")

✅ Found Sentinel-2 image with bands: ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12']

Shape of the new array: (45, 44, 12)
This shape matches the model's expected input of 12 channels.


In [8]:
# --- Assume 'numpy_array' is your image data ---

import numpy as np
from google.colab import files

# Save the array to a binary .npy file
file_name = 'sentinel_image_v2.npy'
np.save(file_name, numpy_array_12_band)

# Trigger a browser download for the saved file
files.download(file_name)

print(f"✅ Saved array to '{file_name}' and started download.")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Saved array to 'sentinel_image_v2.npy' and started download.
